In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/mnist/")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
# Model Parameters
learning_rate = 0.001
n_epochs = 20
batch_size = 100
display_step = 1

In [4]:
# Network Parameters
nd_input = 784
nd_hidden_l1 = 512
nd_hidden_l2 = 256
nd_output = 128

In [5]:
with tf.name_scope("WB") as scope:

    weights = {
        'encoder_w1': tf.Variable(tf.random_normal([nd_input, nd_hidden_l1]),"EN_W1"),
        'encoder_w2': tf.Variable(tf.random_normal([nd_hidden_l1, nd_hidden_l2]),"EN_W2"),
        'encoder_w3': tf.Variable(tf.random_normal([nd_hidden_l2, nd_output]),"EN_W3"),

        'decoder_w1': tf.Variable(tf.random_normal([nd_output, nd_hidden_l2]),"DC_W1"),
        'decoder_w2': tf.Variable(tf.random_normal([nd_hidden_l2, nd_hidden_l1]),"DC_W2"),
        'decoder_w3': tf.Variable(tf.random_normal([nd_hidden_l1, nd_input]),"DC_W3"),
    }

    biases = {
        'encoder_b1': tf.Variable(tf.random_normal([nd_hidden_l1]),"EN_B1"),
        'encoder_b2': tf.Variable(tf.random_normal([nd_hidden_l2]),"EN_B2"),
        'encoder_b3': tf.Variable(tf.random_normal([nd_output]),"EN_B3"),

        'decoder_b1': tf.Variable(tf.random_normal([nd_hidden_l2]),"DC_B1"),
        'decoder_b2': tf.Variable(tf.random_normal([nd_hidden_l1]),"DC_B2"),
        'decoder_b3': tf.Variable(tf.random_normal([nd_input]),"DC_B3"),

    }

In [6]:
# tf Graph input
X = tf.placeholder("float", [None, nd_input],"input")

In [7]:
# Encoder
def deep_encoder(x):
    # First layer with sigmoid activation
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_w1']), biases['encoder_b1'],"ENC1"))
    
    # Second layer with sigmoid activation
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_w2']), biases['encoder_b2'],"ENC2"))
    
    # Output layer with sigmoid activation
    out_layer = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_w3']), biases['encoder_b3'],"ENC3"))
    
    return out_layer

# Decoder
def deep_decoder(x):
    
    # First layer with sigmoid activation
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_w1']), biases['decoder_b1'],"DC1"))
    
    # Second layer with sigmoid activation
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_w2']), biases['decoder_b2'],"DC2"))
    
    # Output layer with sigmoid activation
    out_layer = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_w3']), biases['decoder_b3'],"DC3"))
    
    return out_layer

In [8]:
# Build a  model
with tf.name_scope("encoder") as scope:

    deepencoder_model = deep_encoder(X)
    
with tf.name_scope("decoder") as scope:

    deepdecoder_model = deep_decoder(deepencoder_model)
# Encode input data
Xtr_preds = deepdecoder_model

# True labels
Xtr = X

# root Mean squared error
with tf.name_scope("MSE") as scope:
    train_mse = tf.reduce_mean(tf.square(Xtr-Xtr_preds))
    tf.summary.scalar("train_MSE",train_mse)
    
    test_mse = tf.reduce_mean(tf.square(Xtr-Xtr_preds))
    tf.summary.scalar("test_MSE",test_mse)

# RMSprop optimizer to reduce cost function
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(train_mse)

# Initializing the variables
init = tf.global_variables_initializer()

In [9]:
all_summary = tf.summary.merge_all()

In [12]:
with tf.Session() as sess:
    with tf.device("/cpu:0"):

        sess.run(init)
        
        train_writer = tf.summary.FileWriter('tensorboard/train/',sess.graph)
        test_writer = tf.summary.FileWriter('tensorboard/test/',sess.graph)


        n_batches = int(mnist.train.num_examples/batch_size)

        # 20 epochs
        for epoch in range(n_epochs):

            training_avg_loss = 0
            # Run all batches for each epoch
            for i in range(n_batches):
                xtr_batch, ytr_batch = mnist.train.next_batch(batch_size)

                train_summary,_,c = sess.run([all_summary,optimizer,train_mse], feed_dict={X: xtr_batch})

                train_writer.add_summary(train_summary, epoch*n_batches + i)

                training_avg_loss += c / n_batches
                            

            test_summary,tm = sess.run([all_summary,test_mse], feed_dict={X: mnist.test.images})

            test_writer.add_summary(test_summary, epoch)
            # Display logs per each epoch step
            if epoch % display_step == 0:
                print("Epoch:", '%2d' % (epoch+1), "train_mse=", "{:.5f}".format(training_avg_loss),
                     "test_mse=", "{:.5f}".format(tm))          
            

Epoch:  1 train_mse= 0.17149 test_mse= 0.14615
Epoch:  2 train_mse= 0.13876 test_mse= 0.13359
Epoch:  3 train_mse= 0.12853 test_mse= 0.12409
Epoch:  4 train_mse= 0.12168 test_mse= 0.12008
Epoch:  5 train_mse= 0.11817 test_mse= 0.11574
Epoch:  6 train_mse= 0.11368 test_mse= 0.11288
Epoch:  7 train_mse= 0.11140 test_mse= 0.10967
Epoch:  8 train_mse= 0.10887 test_mse= 0.10871
Epoch:  9 train_mse= 0.10786 test_mse= 0.10772
Epoch: 10 train_mse= 0.10667 test_mse= 0.10662
Epoch: 11 train_mse= 0.10575 test_mse= 0.10584
Epoch: 12 train_mse= 0.10469 test_mse= 0.10463
Epoch: 13 train_mse= 0.10354 test_mse= 0.10378
Epoch: 14 train_mse= 0.10272 test_mse= 0.10328
Epoch: 15 train_mse= 0.10206 test_mse= 0.10267
Epoch: 16 train_mse= 0.10145 test_mse= 0.10190
Epoch: 17 train_mse= 0.10081 test_mse= 0.10154
Epoch: 18 train_mse= 0.10031 test_mse= 0.10089
Epoch: 19 train_mse= 0.09958 test_mse= 0.10037
Epoch: 20 train_mse= 0.09908 test_mse= 0.09986
